In [2]:

# Import SparkSession
from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder \
    .appName("Handling Dates in PySpark") \
    .getOrCreate()

25/02/05 06:27:30 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [3]:
# Sending the Data To HDFS Via Notebook

# Create a CSV file
csv_data = """id,date_iso,date_dmy,date_mdy,timestamp
1,2023-01-15,15/01/2023,01/15/2023,2023-01-15 10:30:00
2,2023-05-20,20/05/2023,05/20/2023,2023-05-20 15:45:00
3,InvalidDate,31/02/2023,02/31/2023,InvalidTimestamp
4,,,,
"""

# Save the CSV file
with open("dates_data.csv", "w") as f:
    f.write(csv_data)


In [5]:
!ls *dates_data*

dates_data.csv


In [6]:
!hadoop fs -put dates_data.csv /data/dates_data.csv

In [7]:
!hadoop fs -ls /data/

Found 12 items
-rw-r--r--   2 root hadoop  343317147 2025-02-03 11:53 /data/customers.csv
-rw-r--r--   2 root hadoop       5488 2025-02-03 11:53 /data/customers_100.csv
-rw-r--r--   2 root hadoop   10528211 2025-02-03 11:53 /data/customers_10mb.csv
-rw-r--r--   2 root hadoop    1060750 2025-02-03 11:53 /data/customers_1mb.csv
-rw-r--r--   2 root hadoop  570783961 2025-02-03 11:53 /data/customers_500mb.csv
-rw-r--r--   2 root hadoop        351 2025-02-04 05:35 /data/customers_with_errors.csv
-rw-r--r--   2 root hadoop         23 2025-02-04 14:53 /data/date.csv
-rw-r--r--   2 root hadoop        209 2025-02-05 06:28 /data/dates_data.csv
drwxr-xr-x   - root hadoop          0 2025-02-04 07:53 /data/write_output.csv
drwxr-xr-x   - root hadoop          0 2025-02-04 07:56 /data/write_output_1_part
drwxr-xr-x   - root hadoop          0 2025-02-04 07:59 /data/write_output_1_part_2
drwxr-xr-x   - root hadoop          0 2025-02-04 08:02 /data/write_output_1_part_3


In [ ]:
# from pyspark.sql.types import StructType, StructField, IntegerType, StringType

# # StructType for the schema
# struct_schema = StructType([
#     StructField("id", IntegerType(), nullable=True),
#     StructField("date_iso", StringType(), nullable=True),
#     StructField("date_dmy", StringType(), nullable=True),
#     StructField("date_mdy", StringType(), nullable=True),
#     StructField("timestamp", StringType(), nullable=True)
# ])


In [14]:


# DDL String for the schema
ddl_schema = """
    id INT,
    date_iso DATE,
    date_dmy DATE,
    date_mdy DATE,
    timestamp TIMESTAMP
"""


# Read the CSV file into a DataFrame
df_file = spark.read.option("header", True).schema(ddl_schema).csv("/data/dates_data.csv")



# Show the DataFrame
df_file.show(truncate=False)

+---+----------+--------+--------+-------------------+
|id |date_iso  |date_dmy|date_mdy|timestamp          |
+---+----------+--------+--------+-------------------+
|1  |2023-01-15|null    |null    |2023-01-15 10:30:00|
|2  |2023-05-20|null    |null    |2023-05-20 15:45:00|
|3  |null      |null    |null    |null               |
|4  |null      |null    |null    |null               |
+---+----------+--------+--------+-------------------+



In [16]:
df_file.printSchema()

root
 |-- id: integer (nullable = true)
 |-- date_iso: date (nullable = true)
 |-- date_dmy: date (nullable = true)
 |-- date_mdy: date (nullable = true)
 |-- timestamp: timestamp (nullable = true)



In [8]:
# Sample data with multiple date formats
data = [
    (1, "2023-01-15", "15/01/2023", "01/15/2023", "2023-01-15 10:30:00"),
    (2, "2023-05-20", "20/05/2023", "05/20/2023", "2023-05-20 15:45:00"),
    (3, "InvalidDate", "31/02/2023", "02/31/2023", "InvalidTimestamp"),  # Invalid dates
    (4, None, None, None, None)  # Null values
]

# Define column names
columns = ["id", "date_iso", "date_dmy", "date_mdy", "timestamp"]


# Create DataFrame
df = spark.createDataFrame(data, schema=columns)

# Show the DataFrame
df.show(truncate=False)

+---+-----------+----------+----------+-------------------+
|id |date_iso   |date_dmy  |date_mdy  |timestamp          |
+---+-----------+----------+----------+-------------------+
|1  |2023-01-15 |15/01/2023|01/15/2023|2023-01-15 10:30:00|
|2  |2023-05-20 |20/05/2023|05/20/2023|2023-05-20 15:45:00|
|3  |InvalidDate|31/02/2023|02/31/2023|InvalidTimestamp   |
|4  |null       |null      |null      |null               |
+---+-----------+----------+----------+-------------------+



In [9]:
df.printSchema()

root
 |-- id: long (nullable = true)
 |-- date_iso: string (nullable = true)
 |-- date_dmy: string (nullable = true)
 |-- date_mdy: string (nullable = true)
 |-- timestamp: string (nullable = true)



In [19]:
from pyspark.sql.functions import to_date

df = df\
    .withColumn('parsed_date_iso', to_date(df.date_iso, 'yyyy-MM-dd'))\
    .withColumn('parsed_date_dmy', to_date(df.date_dmy, 'dd/MM/yyyy'))\
    .withColumn('parsed_date_mdy', to_date(df.date_mdy, 'MM/dd/yyyy'))



In [20]:
df.show(truncate=False)

+---+-----------+----------+----------+-------------------+---------------+---------------+---------------+
|id |date_iso   |date_dmy  |date_mdy  |timestamp          |parsed_date_iso|parsed_date_dmy|parsed_date_mdy|
+---+-----------+----------+----------+-------------------+---------------+---------------+---------------+
|1  |2023-01-15 |15/01/2023|01/15/2023|2023-01-15 10:30:00|2023-01-15     |2023-01-15     |2023-01-15     |
|2  |2023-05-20 |20/05/2023|05/20/2023|2023-05-20 15:45:00|2023-05-20     |2023-05-20     |2023-05-20     |
|3  |InvalidDate|31/02/2023|02/31/2023|InvalidTimestamp   |null           |null           |null           |
|4  |null       |null      |null      |null               |null           |null           |null           |
+---+-----------+----------+----------+-------------------+---------------+---------------+---------------+



# Timestamp

In [22]:
from pyspark.sql.functions import to_timestamp,year,month,dayofmonth,hour,minute


df = df.withColumn('parsed_timestamp',to_timestamp(df.timestamp, 'yyyy-MM-dd HH:mm:ss'))
df.show()
df.printSchema()

+---+-----------+----------+----------+-------------------+---------------+---------------+---------------+-------------------+
| id|   date_iso|  date_dmy|  date_mdy|          timestamp|parsed_date_iso|parsed_date_dmy|parsed_date_mdy|   parsed_timestamp|
+---+-----------+----------+----------+-------------------+---------------+---------------+---------------+-------------------+
|  1| 2023-01-15|15/01/2023|01/15/2023|2023-01-15 10:30:00|     2023-01-15|     2023-01-15|     2023-01-15|2023-01-15 10:30:00|
|  2| 2023-05-20|20/05/2023|05/20/2023|2023-05-20 15:45:00|     2023-05-20|     2023-05-20|     2023-05-20|2023-05-20 15:45:00|
|  3|InvalidDate|31/02/2023|02/31/2023|   InvalidTimestamp|           null|           null|           null|               null|
|  4|       null|      null|      null|               null|           null|           null|           null|               null|
+---+-----------+----------+----------+-------------------+---------------+---------------+-------------

In [25]:
df = df\
.withColumn('year',year(df.parsed_timestamp))\
.withColumn('month',month(df.parsed_timestamp))\
.withColumn('day',dayofmonth(df.parsed_timestamp))\
.withColumn('hour',hour(df.parsed_timestamp))\
.withColumn('minute',minute(df.parsed_timestamp))
df.show()

+---+-----------+----------+----------+-------------------+---------------+---------------+---------------+-------------------+----+-----+----+----+------+
| id|   date_iso|  date_dmy|  date_mdy|          timestamp|parsed_date_iso|parsed_date_dmy|parsed_date_mdy|   parsed_timestamp|year|month| day|hour|minute|
+---+-----------+----------+----------+-------------------+---------------+---------------+---------------+-------------------+----+-----+----+----+------+
|  1| 2023-01-15|15/01/2023|01/15/2023|2023-01-15 10:30:00|     2023-01-15|     2023-01-15|     2023-01-15|2023-01-15 10:30:00|2023|    1|  15|  10|    30|
|  2| 2023-05-20|20/05/2023|05/20/2023|2023-05-20 15:45:00|     2023-05-20|     2023-05-20|     2023-05-20|2023-05-20 15:45:00|2023|    5|  20|  15|    45|
|  3|InvalidDate|31/02/2023|02/31/2023|   InvalidTimestamp|           null|           null|           null|               null|null| null|null|null|  null|
|  4|       null|      null|      null|               null|     

In [29]:
from pyspark.sql.functions import datediff

df = df.withColumn('days_dfference' , datediff(df.parsed_date_mdy,df.parsed_date_iso))
df.select('parsed_date_mdy','parsed_date_iso','days_dfference').show(truncate=False)

+---------------+---------------+--------------+
|parsed_date_mdy|parsed_date_iso|days_dfference|
+---------------+---------------+--------------+
|2023-01-15     |2023-01-15     |0             |
|2023-05-20     |2023-05-20     |0             |
|null           |null           |null          |
|null           |null           |null          |
+---------------+---------------+--------------+



In [31]:
spark.stop()